In [118]:
import pandas as pd

data = pd.read_excel("helpothers.xlsx")

In [331]:
import os
import json
import requests
import geopandas as gpd

import numpy as np
import pandas as pd
import plotly.express as px


def read_geojson():
    
    if not os.path.exists("./data/geodata.json"):
        url = "https://geo.datav.aliyun.com/areas_v3/bound/310000_full.json"
        response = requests.get(url)
        versionInfo = response.text
        versionInfoPython = json.loads(versionInfo)
        with open("./data/geodata.json", "w") as out_file:
            json.dump(versionInfoPython, out_file, indent = 4)
            
    else:   
        with open("./data/geodata.json", "r") as read_file:
            versionInfoPython = json.load(read_file)
    
    return versionInfoPython


def preprocess(data):
    
    area_list = [i["properties"]["name"] for i in versionInfoPython["features"]]
    mapper = {"浦东区":"浦东新区", "闸北区区":"闸北区"}
    data["county"] = data["county"].replace(mapper)

    data = data.loc[data["county"].isin(area_list)].reset_index(drop=True)
    data["date"] = pd.to_datetime(data.createdAt)

    data = data[["date", "county", "helpLevel", "type"]]
    for col in ["helpLevel", "type"]:
        for val in pd.unique(data[col]):
            data[col+"_"+val] = data[col].apply(lambda x: 1 if x==val else 0)

    data = data[[x for x in data if x not in ["helpLevel", "type"]]]
    data["total"] = 1
    cols = ['helpLevel_极紧急', 'helpLevel_紧急', 'helpLevel_较急', 'type_重病', 'type_疾病', 'type_物资', 'type_其它', 'type_孕妇', "total"]
    
    
    data_merge = pd.DataFrame([], index = pd.MultiIndex.from_product([set(data["county"]),set(data["date"])],
                                                                 names=["county", "date"])).reset_index()
    
    data = pd.merge(data_merge, data, how='left').fillna(0)   

    return data.groupby("county").resample('3H', on='date').sum().groupby("county").cumsum().reset_index()


def display_choropleth(candidate):
    
    latitude = 31.2
    longitude = 121.5

    source = data_af.copy()
    source["date"] = source["date"].astype(str)
    
    fig = px.choropleth_mapbox(
        data_frame=source,
        geojson=versionInfoPython,
        color=candidate,
        locations="county",
        featureidkey="properties.name",
        mapbox_style="white-bg",
        animation_frame="date",
        # color_continuous_scale='viridis',
        center={"lat": latitude, "lon": longitude},
        zoom=7.8,
        width=1000,
        height=600,
    )

    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    return fig

In [332]:
versionInfoPython = read_geojson()

data_af = preprocess(data)
data_af
#.

# display_choropleth("total")

helpLevel_极紧急  helpLevel_紧急  helpLevel_较急  \
county date                                                             
嘉定区    2022-04-06 15:00:00            0.0           0.0           0.0   
       2022-04-06 18:00:00            0.0           0.0           1.0   
       2022-04-06 21:00:00            0.0           1.0           1.0   
       2022-04-07 00:00:00            0.0           1.0           2.0   
       2022-04-07 03:00:00            0.0           1.0           2.0   
...                                   ...           ...           ...   
黄浦区    2022-04-14 00:00:00           55.0         130.0         106.0   
       2022-04-14 03:00:00           55.0         130.0         106.0   
       2022-04-14 06:00:00           55.0         130.0         106.0   
       2022-04-14 09:00:00           56.0         138.0         110.0   
       2022-04-14 12:00:00           57.0         142.0         110.0   

                            type_重病  type_疾病  type_物资  type_其它  type_孕妇  total  
county date                                                                     
嘉定区    2022-04-06 15:00:00      0.0      0.0      0.0      0.0      0.0    0.0  
       2022-04-06 18:00:00      0.0      0.0      0.0      1.0      0.0    1.0  
       2022-04-06 21:00:00      0.0      1.0      0.0      1.0      0.0    2.0  
       2022-04-07 00:00:00      0.0      1.0      0.0      2.0      0.0    3.0  
       2022-04-07 03:00:00      0.0      1.0      0.0      2.0      0.0    3.0  
...                             ...      ...      ...      ...      ...    ...  
黄浦区    2022-04-14 00:00:00     55.0    127.0     68.0     38.0      3.0  291.0  
       2022-04-14 03:00:00     55.0    127.0     68.0     38.0      3.0  291.0  
       2022-04-14 06:00:00     55.0    127.0     68.0     38.0      3.0  291.0  
       2022-04-14 09:00:00     56.0    135.0     69.0     41.0      3.0  304.0  
       2022-04-14 12:00:00     57.0    139.0     69.0     41.0      3.0  309.0  

[1024 rows x 9 columns]

In [285]:

data_merge

,county,date
0,松江区,2022-04-08 15:00:00
1,松江区,2022-04-07 00:00:00
2,松江区,2022-04-13 09:00:00
3,松江区,2022-04-12 21:00:00
4,松江区,2022-04-10 03:00:00
...,...,...
1019,闵行区,2022-04-09 00:00:00
1020,闵行区,2022-04-08 21:00:00
1021,闵行区,2022-04-09 12:00:00
1022,闵行区,2022-04-07 12:00:00


In [274]:
data_af.index

MultiIndex([('嘉定区', '2022-04-06 18:00:00'),
            ('嘉定区', '2022-04-06 21:00:00'),
            ('嘉定区', '2022-04-07 00:00:00'),
            ('嘉定区', '2022-04-07 03:00:00'),
            ('嘉定区', '2022-04-07 06:00:00'),
            ('嘉定区', '2022-04-07 09:00:00'),
            ('嘉定区', '2022-04-07 12:00:00'),
            ('嘉定区', '2022-04-07 15:00:00'),
            ('嘉定区', '2022-04-07 18:00:00'),
            ('嘉定区', '2022-04-07 21:00:00'),
            ...
            ('黄浦区', '2022-04-13 09:00:00'),
            ('黄浦区', '2022-04-13 12:00:00'),
            ('黄浦区', '2022-04-13 15:00:00'),
            ('黄浦区', '2022-04-13 18:00:00'),
            ('黄浦区', '2022-04-13 21:00:00'),
            ('黄浦区', '2022-04-14 00:00:00'),
            ('黄浦区', '2022-04-14 03:00:00'),
            ('黄浦区', '2022-04-14 06:00:00'),
            ('黄浦区', '2022-04-14 09:00:00'),
            ('黄浦区', '2022-04-14 12:00:00')],
           names=['county', 'date'], length=977)